In [ ]:
import datetime
import time
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
import os, json

In [ ]:
pasta_ca = "AutoridadeCertificadora"
pasta_csrs = "csrs"
pasta_certs = "certificados"

os.makedirs(pasta_ca, exist_ok=True)
os.makedirs(pasta_csrs, exist_ok=True)
os.makedirs(pasta_certs, exist_ok=True)

ficheiro_cert = os.path.join(pasta_ca,"certificado_raiz.pem")
ficheiro_chave = os.path.join(pasta_ca,"chavepriv_raiz.pem")
revogados = os.path.join(pasta_ca,"certificados_revogados.json")

class AutoridadeCertificadora:

    def __init__(self):
        #self.criar_ca_raiz()
        self.chave_priv = serialization.load_pem_private_key(open(ficheiro_chave, "rb").read(), password=None)
        self.cert_raiz = x509.load_pem_x509_certificate(open(ficheiro_cert, "rb").read())
        self.revogados = self.ler_ficheiros_revogados()
        self.issuer = self.cert_raiz.subject

    def criar_ca_raiz(self):
        os.makedirs(pasta_ca, exist_ok=True)
        chave = rsa.generate_private_key(public_exponent=65537, key_size=4096)
        country_name = input("Country Name:")
        state_or_province_name = input("State or Province Name:")
        locality_name = input("Locality Name:")
        organization_name = input("Organization Name:")
        common_name = input("Common Name:")

        subject = issuer = x509.Name([
            x509.NameAttribute(NameOID.COUNTRY_NAME, country_name),
            x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, state_or_province_name),
            x509.NameAttribute(NameOID.LOCALITY_NAME, locality_name),
            x509.NameAttribute(NameOID.ORGANIZATION_NAME, organization_name),
            x509.NameAttribute(NameOID.COMMON_NAME, common_name)])
        
        cert = (
            x509.CertificateBuilder()
            .subject_name(subject)
            .issuer_name(self.issuer)
            .public_key(chave.public_key())
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.datetime.utcnow())
            .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=3650))
            .add_extension(x509.BasicConstraints(ca=True, path_length=None), critical=True)
            .sign(chave, hashes.SHA256())
        )

        with open(ficheiro_chave, "wb") as f:
            f.write(chave.private_bytes(
                serialization.Encoding.PEM,
                serialization.PrivateFormat.PKCS8,
                serialization.NoEncryption()
            ))
        with open(ficheiro_cert, "wb") as f:
            f.write(cert.public_bytes(serialization.Encoding.PEM))
        print("CA criada com sucesso!")

    def assinar_csr(self, caminho_csr, nome_certificado):
        with open(caminho_csr, "rb") as f:
            csr = x509.load_pem_x509_csr(f.read())

        # Verificar assinatura do CSR (simples)
        if not csr.is_signature_valid:
            print("CSR inválido")
            return False

        cert = (
            x509.CertificateBuilder()
            .subject_name(csr.subject)
            .issuer_name(self.issuer)
            .public_key(csr.public_key())
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.datetime.utcnow())
            .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
            .add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True)
            .sign(self.chave_priv, hashes.SHA256())
        )

        caminho_cert_emitido = os.path.join(pasta_certs, nome_certificado)
        with open(caminho_cert_emitido, "wb") as f:
            f.write(cert.public_bytes(serialization.Encoding.PEM))
        print(f"Certificado {nome_certificado} criado com sucesso!")
        return True

    def emitir_certificado(self, nome, guardar=False):
         chave = rsa.generate_private_key(public_exponent=65537, key_size=4096)
         subject = x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, nome)])
         cert = (
            x509.CertificateBuilder()
            .subject_name(subject)
            .issuer_name(self.issuer)
            .public_key(chave.public_key())
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.datetime.utcnow())
            .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=3650))
            .add_extension(x509.BasicConstraints(ca=True, path_length=None), critical=True)
            .sign(self.chave_priv, hashes.SHA256())
         )
         nova_chave_priv = chave.private_bytes(serialization.Encoding.PEM, serialization.PrivateFormat.PKCS8, serialization.NoEncryption())
         novo_cert = cert.public_bytes(serialization.Encoding.PEM)
        
         if guardar:
             with open(f"certificados/{nome}_chave.pem", "wb") as f:
                f.write(nova_chave_priv)
             with open(f"certificados/{nome}_cert.pem", "wb") as f:
                f.write(novo_cert)
                 
         print(f"Certificado para {nome} criada com sucesso!")
         return nova_chave_priv, novo_cert

    def ler_ficheiros_revogados(self):
        if os.path.exists(revogados):
           with open(revogados) as f:
               return set(json.load(f))
        else:
            return set()

    def pem_para_bytes(self,ficheiro_pem):
        with open(ficheiro_pem, "rb") as f:
            f_bytes = f.read()
        return f_bytes

    def revogar_certificado(self, cert):
        if type(cert) == str:
           cert = self.pem_para_bytes(cert)
        cert = x509.load_pem_x509_certificate(cert)
        self.revogados.add(cert.serial_number)
        with open(revogados, "w") as f:
            json.dump(list(self.revogados), f)
        
    def is_revogado(self,cert):
        if type(cert) == str:
           cert = self.pem_para_bytes(cert)
        cert = x509.load_pem_x509_certificate(cert)
        return cert.serial_number in self.revogados

    def is_identidade_valida(self,cert,nome_esperado):
        if type(cert) == str:
           cert = self.pem_para_bytes(cert)
        cert = x509.load_pem_x509_certificate(cert)
        subject = cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value
        return subject == nome_esperado

    def is_autentico(self, cert, cert_CA):
        if type(cert) == str:
           cert = self.pem_para_bytes(cert)
        if type(cert_CA) == str:
           cert_CA = self.pem_para_bytes(cert_CA)
        cert = x509.load_pem_x509_certificate(cert)
        cert_CA = x509.load_pem_x509_certificate(cert_CA)
        ca_pubkey = cert_CA.public_key()
        try:
           # Verifica se o certificado foi assinado pela CA (assinatura válida)
            ca_pubkey.verify(
            cert.signature,
            cert.tbs_certificate_bytes,  # Parte do certificado que foi assinada
            padding.PKCS1v15(),
            cert.signature_hash_algorithm,
            )
            return True
        except Exception as e:
            print(f"Certificado inválido ou não assinado por CA confiável: {e}")
            return False

    def is_valido(self, certificado):
        if type(certificado) == str:
           certificado = self.pem_para_bytes(certificado)
        certificado = x509.load_pem_x509_certificate(certificado)
        agora = datetime.datetime.utcnow()
        return certificado.not_valid_before <= agora <= certificado.not_valid_after
         

In [ ]:
PASTA_CERTS = "certificados"
pasta_csrs = "csrs"
os.makedirs(PASTA_CERTS, exist_ok=True)
os.makedirs(pasta_csrs, exist_ok=True)

ac = AutoridadeCertificadora()

def listar_certificados():
    print("\n→ Certificados emitidos:")
    for f in os.listdir(PASTA_CERTS):
        if f.endswith("_cert.pem"):
            print(f" - {f}")

def listar_certificados_revogados():
    print("\n→ Certificados revogados:")
    encontrados = False
    for f in os.listdir(PASTA_CERTS):
        if f.endswith("_cert.pem"):
            caminho = os.path.join(PASTA_CERTS, f)
            if ac.is_revogado(caminho):
                print(f" - {f}")
                encontrados = True
    if not encontrados:
        print("Nenhum certificado revogado.")

def listar_certificados_validos():
    print("\n→ Certificados válidos:")
    encontrados = False
    for f in os.listdir(PASTA_CERTS):
        if f.endswith("_cert.pem"):
            caminho = os.path.join(PASTA_CERTS, f)
            if not ac.is_revogado(caminho):
                print(f" - {f}")
                encontrados = True
    if not encontrados:
        print("Nenhum certificado válido encontrado.")

def monitorar_e_assinar_csrs():
    print("Procurando CSRs na pasta csrs/ para assinar...")
    try:
        while True:
            csrs = [f for f in os.listdir(pasta_csrs) if f.endswith(".csr")]
            if not csrs:
                cmd = input("Nenhum CSR encontrado. Coloque um CSR na pasta 'csrs'. Digite 'sair' para sair ou Enter para continuar: ").strip().lower()
                if cmd == "sair":
                    print("Encerrando monitoramento de CSRs.")
                    break
                continue

            for csr_file in csrs:
                caminho_csr = os.path.join(pasta_csrs, csr_file)
                nome_base = csr_file[:-4]  # Remove ".csr"
                nome_cert = f"{nome_base}_cert.pem"

                print(f"Assinando CSR {csr_file}...")
                sucesso = ac.assinar_csr(caminho_csr, nome_cert)

                if sucesso:
                    os.remove(caminho_csr)  # Apaga o CSR após assinar

            print("Processamento de CSRs concluído. Aguarde novos CSRs.")
            time.sleep(5)  # Espera 5 segundos antes de continuar
    except KeyboardInterrupt:
        print("\nMonitoramento interrompido pelo usuário.")

def menu():
    global ac
    while True:
        print("\n--- MENU DA AUTORIDADE CERTIFICADORA ---")
        print("1. Criar nova CA raiz")
        print("2. Emitir certificado para utilizador")
        print("3. Listar certificados emitidos")
        print("4. Revogar certificado")
        print("5. Listar Certificados Revogados")
        print("6. Listar Certificados Válidos")
        print("7. Monitorar e assinar CSRs")
        print("8. Sair")
        escolha = input("Escolha uma opção: ")

        if escolha == "1":
            ac.criar_ca_raiz()
            ac = AutoridadeCertificadora()

        elif escolha == "2":
            nome = input("Nome do utilizador: ")
            ac.emitir_certificado(nome, guardar=True)

        elif escolha == "3":
            listar_certificados()

        elif escolha == "4":
            listar_certificados()
            nome_fich = input("Nome do ficheiro do certificado a revogar (ex: user_cert.pem): ")
            caminho = os.path.join(PASTA_CERTS, nome_fich)
            if os.path.exists(caminho):
                ac.revogar_certificado(caminho)
                print("Certificado revogado.")
            else:
                print("Ficheiro não encontrado.")

        elif escolha == "5":
            listar_certificados_revogados()

        elif escolha == "6":
            listar_certificados_validos()

        elif escolha == "7":
            monitorar_e_assinar_csrs()

        elif escolha == "8":
            print("A sair...")
            break

        else:
            print("Opção inválida.")

if __name__ == "__main__":
    menu()
    
